## An Electrode Design Optimisation Example

A design optimisation example loosely based on work by L.D. Couto available at [[1]](https://doi.org/10.1016/j.energy.2022.125966).

The target is to maximise the gravimetric energy density over a range of possible design parameter values, including for example:

cross-sectional area = height x width (only need change one), electrode widths, particle radii, volume fractions and separator width.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm
from pybamm import Parameter

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Optimising the Parameters

First, we define the model to be used for the parameter optimisation,

In [ ]:
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values.update(
    {
        "Electrolyte density [kg.m-3]": Parameter("Separator density [kg.m-3]"),
        "Negative electrode active material density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Negative electrode carbon-binder density [kg.m-3]": Parameter(
            "Negative electrode density [kg.m-3]"
        ),
        "Positive electrode active material density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
        "Positive electrode carbon-binder density [kg.m-3]": Parameter(
            "Positive electrode density [kg.m-3]"
        ),
    },
    check_already_exists=False,
)
model = pybamm.lithium_ion.SPMe()

Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a prior for the parameters. The initial parameters values used in the optimisation will be randomly drawn from the prior distribution.

In [ ]:
parameters = [
    pybop.Parameter(
        "Positive electrode thickness [m]",
        prior=pybop.Gaussian(7.56e-05, 0.05e-05),
        bounds=[65e-06, 10e-05],
    ),
    pybop.Parameter(
        "Positive particle radius [m]",
        prior=pybop.Gaussian(5.22e-06, 0.05e-06),
        bounds=[2e-06, 9e-06],
    ),
]

Next, we construct the experiment for design optimisation and the initial state-of-charge,

In [ ]:
experiment = pybamm.Experiment(["Discharge at 1C until 2.5 V (5 seconds period)"])
initial_soc = 0.7

We can now define the builder with the model and the cost function, which in this example is the `GravimetricEnergyDensity()` used to maximise the gravimetric energy density of the cell. We then use the builder to construct the design optimisation problem.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_simulation(
        model,
        parameter_values=parameter_values,
        experiment=experiment,
        initial_state=initial_soc,
    )
    .add_cost(pybop.costs.pybamm.GravimetricEnergyDensity())
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

Next we construct an optimiser. This class provides the algorithm to optimise the parameters. For this example, we use particle swarm optimisation (PSO). Due to the computational requirements of the design optimisation methods, we limit the number of iterations for this example.

In [ ]:
options = pybop.PintsOptions(
    verbose=True,
    max_iterations=15,
)
optim = pybop.PSO(problem, options=options)

Finally, we run the optimisation and return the values obtained,

In [ ]:
results = optim.run()
print(f"Initial gravimetric energy density: {-results.initial_cost:.2f} Wh.kg-1")
print(f"Optimised gravimetric energy density: {-results.best_cost:.2f} Wh.kg-1")

Iter: 1 | Evals: 6 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 2 | Evals: 12 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 3 | Evals: 18 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 4 | Evals: 24 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 5 | Evals: 30 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


...

Iter: 7 | Evals: 42 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 8 | Evals: 48 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 9 | Evals: 54 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


Iter: 10 | Evals: 60 | Best Values: [7.31454915e-05 2.07979163e-06] | Best Cost: -273.04355200821976 |


OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [7.56456024e-05 5.27456414e-06]
  Optimised parameters: [7.31454915e-05 2.07979163e-06]
  Diagonal Fisher Information entries: None
  Best cost: -273.04355200821976
  Optimisation time: 57.62549304962158 seconds
  Number of iterations: 15
  Number of evaluations: 90
  Reason for stopping: Maximum number of iterations (15) reached.
Initial gravimetric energy density: 254.38 Wh.kg-1
Optimised gravimetric energy density: 273.04 Wh.kg-1


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

We can plot the model predictions using the estimated parameters compared to the initial parameters.

In [ ]:
sim1 = pybamm.Simulation(
    model, parameter_values=parameter_values, experiment=experiment
)
sol1 = sim1.solve(initial_soc=initial_soc)

sim2 = pybamm.Simulation(
    model, parameter_values=results.parameter_values, experiment=experiment
)
sol2 = sim2.solve(initial_soc=initial_soc)

pybop.plot.trajectories(
    x=[sol1["Time [s]"].data, sol2["Time [s]"].data],
    y=[sol1["Voltage [V]"].data, sol2["Voltage [V]"].data],
    trace_names=["Initial parameters", "Optimised parameters"],
    xaxis_title="Time / s",
    yaxis_title="Voltage / V",
);

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:


In [ ]:
pybop.plot.surface(optim);